In [ ]:
"""
# Command line
D:
cd Backup\fintech-assignments\AML\stanford-corenlp-full-2018-10-05

english
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -preload tokenize,ssplit,pos,lemma,ner,parse,depparse -status_port 9000 -port 9000 -timeout 50000 & 

chinese:
java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -serverProperties StanfordCoreNLP-chinese.properties -preload tokenize,ssplit,pos,lemma,ner,parse -status_port 9001  -port 9001 -timeout 50000


# Set environment variable
os.environ['STANFORD_MODELS'] = './stanford-corenlp-full-2018-10-05'
os.environ['CLASSPATH'] = './stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2'
os.environ["JAVA_HOME"] = r'D:\Java\jre1.8.0_211'
"""

In [1]:
import os
import re 
import jieba
import pandas as pd
from opencc import OpenCC
from nltk.parse import CoreNLPParser
from nltk.tree import Tree

In [3]:
df = pd.read_csv(os.path.join('crawl_data_csv', 'news_data_with_PER.csv'))
df.head()

,news,keywords,title,link,content,content_cn,PER
0,appledaily,人口販運,台灣捲入跨國人口販運風暴｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/appledaily/articl...,《報導者》獨家授權記者╱《報導者》蔣宜婷、李雪莉 台北─柬埔寨報導台灣已捲入遠洋漁業的人口販...,《报导者》独家授权记者╱《报导者》蒋宜婷、李雪莉 台北─柬埔寨报导台湾已卷入远洋渔业的人口贩...,"蒋宜婷,李雪莉"
1,appledaily,人口販運,​內政部：今年1到9月查緝人口販運被害人安置印尼人最多｜蘋果新聞網 ...,http://www.appledaily.com.tw/realtimenews/arti...,內政部公佈統計數據指出，今年1到9月司法及警察機關查緝人口販運案件共116件，75%為性剝削...,内政部公布统计数据指出，今年1到9月司法及警察机关查缉人口贩运案件共116件，75%为性剥削...,张文馨
2,appledaily,人口販運,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,内政部「2018防制人口贩运国际工作坊」今（25）日盛大揭幕，邀请副总统陈建仁、美国在台协会...,"陈建仁,谷立言,陈建仁,谷立言,罗秉成,徐国勇,Jean,Patrick Taran,吴玉琴..."
3,appledaily,人口販運,赴美留學成奴隸！洛城破獲大宗人口販賣｜蘋果新聞網｜蘋果日報https://,https://www.appledaily.com.tw/realtimenews/art...,（新增配音影片）洛縣人口販賣執行組周二宣佈，搗破一宗全國性人口販賣案，3日的掃蕩行動中拘捕5...,（新增配音影片）洛县人口贩卖执行组周二宣布，捣破一宗全国性人口贩卖案，3日的扫荡行动中拘捕5...,"伍国庆,张紫茵,洛杉矶"
4,appledaily,人口販運,美人口販運報告台列最佳等級、中國最差｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,美國國務院昨日公布年度人口販運問題報告，台灣連續第9年名列最佳第一級國家。中國則被列為人口販...,美国国务院昨日公布年度人口贩运问题报告，台湾连续第9年名列最佳第一级国家。中国则被列为人口贩...,"史密斯,庞皮欧"


In [4]:
low_risk_keywords = ["人口販運", "性剝削", "偽造貨幣", "殺人", "重傷害", "搶奪", "勒贖", "海盜", "恐怖主義", "資恐"]
medium_risk_keywords = ["非法販賣武器", "贓物", "竊盜", "綁架", "拘禁", "妨害自由", "環保犯罪", "偽造文書"]
high_risk_keywords = ["仿冒", "盜版", "侵害營業秘密"]
exhigh_risk_keywords = ["毒品販運", "詐欺", "走私", "稅務犯罪", "組織犯罪", "證券犯罪", "貪汙賄賂", "第三方洗錢"]

def risk_level(df):
    crime = df['keywords']
    if crime in low_risk_keywords: risk = '低'
    elif crime in medium_risk_keywords: risk = '中'
    elif crime in high_risk_keywords: risk = '高'
    elif crime in exhigh_risk_keywords: risk = '非常高'
    else: risk = float('nan')
    return risk

df['crime_risk'] = df.apply(risk_level, axis=1)
df.head()

,news,keywords,title,link,content,content_cn,PER,crime_risk
0,appledaily,人口販運,台灣捲入跨國人口販運風暴｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/appledaily/articl...,《報導者》獨家授權記者╱《報導者》蔣宜婷、李雪莉 台北─柬埔寨報導台灣已捲入遠洋漁業的人口販...,《报导者》独家授权记者╱《报导者》蒋宜婷、李雪莉 台北─柬埔寨报导台湾已卷入远洋渔业的人口贩...,"蒋宜婷,李雪莉",低
1,appledaily,人口販運,​內政部：今年1到9月查緝人口販運被害人安置印尼人最多｜蘋果新聞網 ...,http://www.appledaily.com.tw/realtimenews/arti...,內政部公佈統計數據指出，今年1到9月司法及警察機關查緝人口販運案件共116件，75%為性剝削...,内政部公布统计数据指出，今年1到9月司法及警察机关查缉人口贩运案件共116件，75%为性剥削...,张文馨,低
2,appledaily,人口販運,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,内政部「2018防制人口贩运国际工作坊」今（25）日盛大揭幕，邀请副总统陈建仁、美国在台协会...,"陈建仁,谷立言,陈建仁,谷立言,罗秉成,徐国勇,Jean,Patrick Taran,吴玉琴...",低
3,appledaily,人口販運,赴美留學成奴隸！洛城破獲大宗人口販賣｜蘋果新聞網｜蘋果日報https://,https://www.appledaily.com.tw/realtimenews/art...,（新增配音影片）洛縣人口販賣執行組周二宣佈，搗破一宗全國性人口販賣案，3日的掃蕩行動中拘捕5...,（新增配音影片）洛县人口贩卖执行组周二宣布，捣破一宗全国性人口贩卖案，3日的扫荡行动中拘捕5...,"伍国庆,张紫茵,洛杉矶",低
4,appledaily,人口販運,美人口販運報告台列最佳等級、中國最差｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,美國國務院昨日公布年度人口販運問題報告，台灣連續第9年名列最佳第一級國家。中國則被列為人口販...,美国国务院昨日公布年度人口贩运问题报告，台湾连续第9年名列最佳第一级国家。中国则被列为人口贩...,"史密斯,庞皮欧",低


In [5]:
t2s = OpenCC('t2s')
s2t = OpenCC('s2t')
df.rename(columns = {'PER': 'PER_cn'}, inplace = True)
def s2t_convert(series):
    if type(series) == str: return s2t.convert(series)
    else: return series
df['PER'] = df['PER_cn'].apply(s2t_convert)

In [16]:
def replace_str(series):
    PER_list = []
    if type(series) == str:
        for name in series.split(','):
            for char in ['【', '】', '「', '」', '+', '?', '`？', '﹖', '（', '）', '(', ')', '〔', '〕', '╱', '★', '.', '。', ':', '：', '=']:
                name = name.replace(char, '')
            if name.replace(' ', '') != '': PER_list.append(name)
        PER = ','.join(PER_list)
        if PER == []: return float('nan')
        else: return PER
    else: return series
    
df['PER'] = df['PER'].apply(replace_str)
df['content'] = df['content'].apply(lambda content: content.replace('\n', '。'))
df.dropna(subset = ['content', 'PER'], how = 'any', inplace = True)
df.head()

,news,keywords,title,link,content,content_cn,PER_cn,crime_risk,PER
0,appledaily,人口販運,台灣捲入跨國人口販運風暴｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/appledaily/articl...,《報導者》獨家授權記者╱《報導者》蔣宜婷、李雪莉 台北─柬埔寨報導台灣已捲入遠洋漁業的人口販...,《报导者》独家授权记者╱《报导者》蒋宜婷、李雪莉 台北─柬埔寨报导台湾已卷入远洋渔业的人口贩...,"蒋宜婷,李雪莉",低,"蔣宜婷,李雪莉"
1,appledaily,人口販運,​內政部：今年1到9月查緝人口販運被害人安置印尼人最多｜蘋果新聞網 ...,http://www.appledaily.com.tw/realtimenews/arti...,內政部公佈統計數據指出，今年1到9月司法及警察機關查緝人口販運案件共116件，75%為性剝削...,内政部公布统计数据指出，今年1到9月司法及警察机关查缉人口贩运案件共116件，75%为性剥削...,张文馨,低,張文馨
2,appledaily,人口販運,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,内政部「2018防制人口贩运国际工作坊」今（25）日盛大揭幕，邀请副总统陈建仁、美国在台协会...,"陈建仁,谷立言,陈建仁,谷立言,罗秉成,徐国勇,Jean,Patrick Taran,吴玉琴...",低,"陳建仁,谷立言,陳建仁,谷立言,羅秉成,徐國勇,Jean,Patrick Taran,吳玉琴..."
3,appledaily,人口販運,赴美留學成奴隸！洛城破獲大宗人口販賣｜蘋果新聞網｜蘋果日報https://,https://www.appledaily.com.tw/realtimenews/art...,（新增配音影片）洛縣人口販賣執行組周二宣佈，搗破一宗全國性人口販賣案，3日的掃蕩行動中拘捕5...,（新增配音影片）洛县人口贩卖执行组周二宣布，捣破一宗全国性人口贩卖案，3日的扫荡行动中拘捕5...,"伍国庆,张紫茵,洛杉矶",低,"伍國慶,張紫茵,洛杉磯"
4,appledaily,人口販運,美人口販運報告台列最佳等級、中國最差｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,美國國務院昨日公布年度人口販運問題報告，台灣連續第9年名列最佳第一級國家。中國則被列為人口販...,美国国务院昨日公布年度人口贩运问题报告，台湾连续第9年名列最佳第一级国家。中国则被列为人口贩...,"史密斯,庞皮欧",低,"史密斯,龐皮歐"


In [23]:
df = df.drop_duplicates(keep = 'first')
df.reset_index(drop = True, inplace = True)
df.head()

,news,keywords,title,link,content,content_cn,PER_cn,crime_risk,PER
0,appledaily,人口販運,台灣捲入跨國人口販運風暴｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/appledaily/articl...,《報導者》獨家授權記者╱《報導者》蔣宜婷、李雪莉 台北─柬埔寨報導台灣已捲入遠洋漁業的人口販...,《报导者》独家授权记者╱《报导者》蒋宜婷、李雪莉 台北─柬埔寨报导台湾已卷入远洋渔业的人口贩...,"蒋宜婷,李雪莉",低,"蔣宜婷,李雪莉"
1,appledaily,人口販運,​內政部：今年1到9月查緝人口販運被害人安置印尼人最多｜蘋果新聞網 ...,http://www.appledaily.com.tw/realtimenews/arti...,內政部公佈統計數據指出，今年1到9月司法及警察機關查緝人口販運案件共116件，75%為性剝削...,内政部公布统计数据指出，今年1到9月司法及警察机关查缉人口贩运案件共116件，75%为性剥削...,张文馨,低,張文馨
2,appledaily,人口販運,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,内政部「2018防制人口贩运国际工作坊」今（25）日盛大揭幕，邀请副总统陈建仁、美国在台协会...,"陈建仁,谷立言,陈建仁,谷立言,罗秉成,徐国勇,Jean,Patrick Taran,吴玉琴...",低,"陳建仁,谷立言,陳建仁,谷立言,羅秉成,徐國勇,Jean,Patrick Taran,吳玉琴..."
3,appledaily,人口販運,赴美留學成奴隸！洛城破獲大宗人口販賣｜蘋果新聞網｜蘋果日報https://,https://www.appledaily.com.tw/realtimenews/art...,（新增配音影片）洛縣人口販賣執行組周二宣佈，搗破一宗全國性人口販賣案，3日的掃蕩行動中拘捕5...,（新增配音影片）洛县人口贩卖执行组周二宣布，捣破一宗全国性人口贩卖案，3日的扫荡行动中拘捕5...,"伍国庆,张紫茵,洛杉矶",低,"伍國慶,張紫茵,洛杉磯"
4,appledaily,人口販運,美人口販運報告台列最佳等級、中國最差｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,美國國務院昨日公布年度人口販運問題報告，台灣連續第9年名列最佳第一級國家。中國則被列為人口販...,美国国务院昨日公布年度人口贩运问题报告，台湾连续第9年名列最佳第一级国家。中国则被列为人口贩...,"史密斯,庞皮欧",低,"史密斯,龐皮歐"


In [24]:
df.to_csv(r'D:\Backup\fintech-assignments\AML\final_demo\news_data.csv')

In [8]:
def find_sentence(key, text, end_list = ['。', '！', '!', '？', '?']):
    for oc in re.finditer(key, text):
        sentence_start = 0
        sentence_end = len(text) - 1
        for end in end_list:
            dis_to_occurstart = text[:oc.start()][::-1].find(end)
            if dis_to_occurstart != -1: sentence_start = max(sentence_start, oc.start() - dis_to_occurstart) 
            dis_to_occurend = text[oc.end():].find(end)
            if dis_to_occurend != -1: sentence_end = min(sentence_end, oc.end() + dis_to_occurend)   
        sentence = text[sentence_start: sentence_end+1]
        
        if 'sentences' in locals(): 
            if sentences.find(sentence) == -1: sentences = sentences + '\n' + sentence
        else: sentences = sentence
    
    if 'sentences' in locals(): return sentences
    else: return float('nan')

In [9]:
def real_suspect(name, text):
    suspect_noun = ['嫌犯', '疑犯', '嫌疑犯', '主嫌', '犯人', '共犯']
    suspect_active_verb = ['涉嫌', '犯下', '犯案', '觸法', '違法', '觸犯']
    suspect_passive_verb = ['逮捕', '逮到', '拘捕', '居留', '查緝', '傳喚']
    
    is_suspect = False
    try:
        for n in re.finditer(name, text):
            if is_suspect == False: 
                for sn in suspect_noun:
                    if (text[n.start()-len(sn): n.start()] == sn) | (text[n.end(): n.end()+len(sn)] == sn): 
                        is_suspect = True
                        break
            if is_suspect == False: 
                for sav in suspect_active_verb:
                    if (text[n.end(): n.end()+len(sav)] == sav): 
                        is_suspect = True
                        break
            if is_suspect == False: 
                for spv in suspect_passive_verb:
                    if (text[n.start()-len(spv): n.start()] == spv): 
                        is_suspect = True
                        break
    except: pass    
    return is_suspect

In [10]:
def not_suspect(name, text):
    delete_list = ['記者', '特派員']
    no_suspect = False
    try:
        for n in re.finditer(name, text):
            if no_suspect == False: 
                for d in delete_list:
                    if (text[n.start()-len(d): n.start()] == d) | (text[n.end(): n.end()+len(d)] == d): 
                        no_suspect = True
                        break
    except: pass
    return no_suspect

In [11]:
def mindis_suspect(name, crime, text, dis_dict):
    # Find minimal distance of the name and the keyword in a sentence.
    try:
        for n in re.finditer(name, text):
            for c in re.finditer(crime, text):
                if n.end()-1 < c.start(): dis = c.start() - n.end() + 1
                else: dis = n.start() - c.end() + 1
                if name in dis_dict.keys():
                    if dis < dis_dict[name]: dis_dict[name] = dis
                else: dis_dict[name] = dis
    except: pass
    return dis_dict

In [30]:
def find_suspect(df):
    PER_set = set(df['PER'].split(','))
    content = df['content']
    crime = df['keywords']
    crime_risk = df['crime_risk']
    news = df['news']
    title = df['title']
    link = df['link']
    
    name_list = []
    suspect_prob_list = []
    key_sentence_list = []
    other_name_list = []
    
    for name in PER_set:
        is_suspect = real_suspect(name, content)
        if is_suspect == True: 
            name_list.append(name)
            suspect_prob_list.append('高')
    
    suspect_amount = len(name_list)
    if suspect_amount == 0:
        # Find sentence with crime.
        sentences_with_crime = find_sentence(crime, content)
        if type(sentences_with_crime) == str: 
            sentences_with_crime_list = sentences_with_crime.split('\n')
            for sentence in sentences_with_crime_list:   
                dis_dict = {}
                for name in PER_set:
                    no_suspect = not_suspect(name, content)
                    if no_suspect == False:
                        # Find minimal distance of the name and the keyword in a sentence.
                        dis_dict = mindis_suspect(name, crime, sentence, dis_dict)
                if len(dis_dict.values()) != 0: 
                    mindis = min(dis_dict.values()) 
                    name_with_mindis = [list(dis_dict.keys())[i] for i, x in enumerate(dis_dict.values()) if x == mindis]
                    name_list += name_with_mindis            
            name_list = list(set(name_list))
            suspect_amount += len(name_list)
            suspect_prob_list += ['低']*len(name_list)
    
    if suspect_amount != 0:
        for name in name_list:
            key_sentence = find_sentence(name, content)
            key_sentence_list.append(key_sentence)
        crime_list = [crime]*suspect_amount
        crime_risk_list = [crime_risk]*suspect_amount
        news_list = [news]*suspect_amount
        title_list = [title]*suspect_amount
        link_list = [link]*suspect_amount
        other_suspect_list = []
        for name in name_list:
            temp = name_list.copy()
            temp.remove(name)
            if temp != []: other_suspect_list.append( ', '.join(temp))
            else: other_suspect_list.append(float('nan'))

        Final_demo = pd.DataFrame({'name': name_list, 'crime': crime_list, 'crime_risk': crime_risk_list,\
                                   'suspect_probability': suspect_prob_list, 'key_sentence': key_sentence_list,\
                                   'news': news_list, 'news_title': title_list, 'news_link': link_list,\
                                   'other_suspect': other_suspect_list})
    else: Final_demo = pd.DataFrame()
    return Final_demo

df_suspect = pd.concat(list(df.apply(find_suspect, axis=1)))
df_suspect.reset_index(drop = True, inplace = True)
df_suspect.head()

,name,crime,crime_risk,suspect_probability,key_sentence,news,news_title,news_link,other_suspect
0,李雪莉,人口販運,低,低,《報導者》獨家授權記者╱《報導者》蔣宜婷、李雪莉 台北─柬埔寨報導台灣已捲入遠洋漁業的人口販...,appledaily,台灣捲入跨國人口販運風暴｜蘋果新聞網｜蘋果日報,http://www.appledaily.com.tw/appledaily/articl...,NaN
1,蔡,人口販運,低,低,副總統陳建仁致詞表示，蔡總統於96年成立「防制人口販運協調會報」聯繫平台，12年來各部會整合...,appledaily,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,"徐國勇, 陳建仁, 谷立言, 李麗芬"
2,徐國勇,人口販運,低,低,今天活動由副總統陳建仁、美國在台協會代理處長谷立言、行政院政務委員羅秉成、內政部長徐國勇、美...,appledaily,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,"蔡, 陳建仁, 谷立言, 李麗芬"
3,陳建仁,人口販運,低,低,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,appledaily,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,"蔡, 徐國勇, 谷立言, 李麗芬"
4,谷立言,人口販運,低,低,內政部「2018防制人口販運國際工作坊」今（25）日盛大揭幕，邀請副總統陳建仁、美國在台協會...,appledaily,打擊人口販運台成立防制人口販運國際工作坊| 蘋果日報,http://www.appledaily.com.tw/realtimenews/arti...,"蔡, 徐國勇, 陳建仁, 李麗芬"


In [31]:
# Final demo dataframe
df_suspect.to_csv(r'D:\Backup\fintech-assignments\AML\final_demo\AML_data.csv')

In [ ]:
################################################################################################################################

In [7]:
parser = CoreNLPParser(url = 'http://localhost:9001', tagtype = 'ner')   # tagtype = 'pos' or 'ner'

In [63]:
def find_occupation(df):
    PER_set = set(df['PER'].split(','))
    content = df['content']
    for name in PER_set:
        try:
            for n in re.finditer(name, content):
                end_list = ['。', '.', '！', '!', '？', '?', '，', ',', '；', '；', '、']
                sentence_start = 0
                sentence_end = len(content) - 1
                for end in end_list:
                    dis_to_nstart = content[:n.start()][::-1].find(end)
                    if dis_to_nstart != -1: sentence_start = max(sentence_start, n.start() - dis_to_nstart) 
                    dis_to_nend = content[n.end():].find(end)
                    if dis_to_nend != -1: sentence_end = min(sentence_end, n.end() + dis_to_nend)    
                sentence_front = content[sentence_start: n.start()] 
                sentence_behind = content[n.end(): sentence_end+1]
                cut_front = list(parser.tokenize(sentence_front))  
                cut_behind = list(parser.tokenize(sentence_behind))  
                nltk_cut = cut_front + [name]
                nltk_tree = list(parser.parse(nltk_cut))[0]
                words = list(w for w in nltk_tree.leaves())
                position = words.index(name)
                treeposition = nltk_tree.leaf_treeposition(position)
                for i in range(len(treeposition)):
                    label = nltk_tree[treeposition[0:i+1]].label()
                    if label == 'NP':
                        occupation = ''.join(nltk_tree[treeposition[: i+1]].leaves())[0:-len(name)]
                        print(name, occupation, sentence_front + name)
                        break
        except:  pass
df.iloc[0:50].apply(find_occupation, axis=1)

蔣宜婷 《報導者》 《報導者》獨家授權記者╱《報導者》蔣宜婷
徐國勇 內政部長 內政部長徐國勇
徐國勇 內政部長 內政部長徐國勇
Jean  美國自由聯盟執行長Jean
劉文淵 突發中心 （突發中心劉文淵
吳玉琴 立委 全球移民政策協會理事長Patrick Taran與立委吳玉琴
傅德恩 處長 與會的美國在台協會(AIT)代理處長傅德恩
李麗華 宜蘭縣漁工職業工會秘書長 宜蘭縣漁工職業工會秘書長李麗華
川普 Tillerson）及 美國務卿提勒森（Rex Tillerson）及川普
提勒森 美國務卿 美國務卿提勒森
蔡英文 台灣列最佳等級國家 （陳培煌/台北報導）【相關新聞】美防制人口販運有功獎項　首度有台灣人獲獎台灣列最佳等級國家　蔡英文
陳培煌  （陳培煌
李麗華 宜蘭縣漁工職業工會的秘書長兼創辦人 宜蘭縣漁工職業工會的秘書長兼創辦人李麗華
柯佩吉 ‧ 由打擊人口販運問題大使蘇珊‧柯佩吉
蘇珊 大使 由打擊人口販運問題大使蘇珊
李德揚 南加州台灣旅館同業公會會長 」南加州台灣旅館同業公會會長李德揚
陳志豪  （陳志豪
洛杉磯 陳志豪／ （陳志豪／洛杉磯
卡特蘭 Royce）幕僚長  Royce）幕僚長卡特蘭
提勒森 美國國務卿  美國國務卿提勒森
傅德恩 處長 美國在台協會台北辦事處代理處長傅德恩
敦義  吳副總統敦義
敦義 吳 吳敦義
莫天虎 敬及移民署長 並由內政部次長陳純敬及移民署長莫天虎
李定宇  （李定宇
張榮興   警察局局長張榮興
林瑞麟 移民署台中市專勤隊分隊長 移民署台中市專勤隊分隊長林瑞麟
蔡女 老鴇 抓到老鴇蔡女
楊勝裕 突發中心 （突發中心楊勝裕
蔡 老鴇 抓到老鴇蔡
周文如 新竹檢察官 新竹檢察官周文如
周文如  並報新竹地檢署周文如
蘇男 楊女及 楊女及蘇男
蘇男 判處 判處蘇男
鄭男 護 工作內容應該為看護鄭男
鄭男 但 但鄭男
鄭男  並將A1引介到鄭男
鄭男  依違反《人口販運防制法》等罪起訴鄭男
顏凡裴  （顏凡裴
蔡女利 鄭男與 鄭男與蔡女利
林全今  行政院長林全今
林萬億  請政務委員林萬億
陳 新北市一名 新北市一名陳
陳 一舉將 一舉將陳
陳男 將 將陳男
陳男  都是陳男
林 樹 樹林
林 嘉義憲兵隊及移民署兵分四路至樹 嘉義憲兵隊及移民署兵分四路至樹林
林 嫌及 一舉將陳姓犯嫌及林
林 與 陳姓犯嫌與林
林 嫌與

0     None
2     None
5     None
7     None
9     None
11    None
13    None
14    None
16    None
17    None
19    None
20    None
21    None
27    None
28    None
29    None
31    None
35    None
37    None
38    None
39    None
44    None
45    None
47    None
49    None
56    None
58    None
60    None
61    None
62    None
63    None
67    None
70    None
72    None
73    None
74    None
75    None
76    None
80    None
82    None
83    None
85    None
86    None
87    None
88    None
90    None
94    None
95    None
96    None
97    None
dtype: object

In [60]:
def find_verb(df):
    PER_set = set(df['PER'].split(','))
    content = df['content']
    for name in PER_set:
        try:
            for n in re.finditer(name, content):
                end_list = ['。', '.', '！', '!', '？', '?', '，', ',', '；', '；']
                sentence_start = 0
                sentence_end = len(content) - 1
                for end in end_list:
                    dis_to_nstart = content[:n.start()][::-1].find(end)
                    if dis_to_nstart != -1: sentence_start = max(sentence_start, n.start() - dis_to_nstart) 
                    dis_to_nend = content[n.end():].find(end)
                    if dis_to_nend != -1: sentence_end = min(sentence_end, n.end() + dis_to_nend)    
                sentence_front = content[sentence_start: n.start()] 
                sentence_behind = content[n.end(): sentence_end+1]
                cut_front = list(parser.tokenize(sentence_front)) + [name]  
                cut_behind = [name] + list(parser.tokenize(sentence_behind))  
                tree_front = list(parser.parse(cut_front))[0]
                tree_behind = list(parser.parse(cut_behind))[0]
                words = list(w for w in tree_behind.leaves())
                position = words.index(name)
                treeposition = tree_behind.leaf_treeposition(position)
                print(tree_behind)
                for subtree in tree_behind.subtrees():
                    if subtree.label() == 'VP':
                        print(subtree.leaves())
                        break
        except: pass

df.iloc[0:5].apply(find_verb, axis=1)

(ROOT
  (IP
    (NP (NR 蔣宜婷) (PU 、) (NR 李雪莉) (NR 台北))
    (VP
      (VP
        (VV ─)
        (NP
          (CP
            (IP
              (NP (NP (NP (NR 柬埔寨)) (NP (NN 報導))) (NP (NR 台灣)))
              (VP (ADVP (AD 已)) (VP (VV 捲入遠洋漁業))))
            (DEC 的))
          (NP (NN 人口) (NN 販運))))
      (VP (VV 風暴)))
    (PU 。)))
['─', '柬埔寨', '報導', '台灣', '已', '捲入遠洋漁業', '的', '人口', '販運', '風暴']
(ROOT
  (IP
    (NP (NR 李雪莉) (NR 台北))
    (VP
      (VP
        (VV ─)
        (NP
          (CP
            (IP
              (NP (NP (NP (NR 柬埔寨)) (NP (NN 報導))) (NP (NR 台灣)))
              (VP (ADVP (AD 已)) (VP (VV 捲入遠洋漁業))))
            (DEC 的))
          (NP (NN 人口) (NN 販運))))
      (VP (VV 風暴)))
    (PU 。)))
['─', '柬埔寨', '報導', '台灣', '已', '捲入遠洋漁業', '的', '人口', '販運', '風暴']
(ROOT
  (IP
    (NP (NR 徐國勇) (PU 、) (NR 美國) (NN 自由) (NN 聯盟執))
    (VP
      (VV 行長)
      (IP
        (NP
          (NP
            (NP
              (NP
                (NR Jean)
                (NR Bruggeman)
                (

0    None
2    None
5    None
7    None
9    None
dtype: object

In [ ]:
################################################################################################################################

In [ ]:
def Tag(nltk_cut):
    nltk_tag = list(parser.tag(nltk_cut))
    # print('nltk_tag:', nltk_tag, '\n')

    per = []
    org = []
    loc = []
    for wordtag_pair in nltk_tag:
        index = nltk_tag.index(wordtag_pair)

        word = wordtag_pair[0]
        tag = wordtag_pair[1]
        if tag == 'PERSON':
            if nltk_tag[index-1][1] == 'PERSON': per[-1] += word
            else: per.append(word)

        if tag == 'ORGANIZATION':
            if nltk_tag[index-1][1] == 'ORGANIZATION': org[-1] += word
            else: org.append(word)

        if tag == 'GPE':
            if nltk_tag[index-1][1] == 'GPE': loc[-1] += word
            else: loc.append(word)    
    
    # print(per, '\n', org, '\n', loc, '\n')
    return per, org, loc

In [ ]:
for line in nltk_sentence:
    for sentence in line:
        sentence.draw()

In [13]:
s = '全球最大電信網路設備製造商之一華為的首席財務官CFO孟晚舟於當地時間12月1日在加拿大被捕'
nltk_cut = list(parser.tokenize(s))  
nltk_tree = list(parser.parse(nltk_cut))[0]

In [15]:
tree_front = list(parser.parse(list(parser.tokenize('全球最大電信網路設備製造商之一華為的首席財務官CFO'))))[0]
for subtree in nltk_tree.subtrees():
    if subtree.label() == 'IP':
        print(subtree.leaves())

['全球', '最大', '電信', '網路設', '備製', '造商', '之一', '華為', '的', '首席', '財務官', 'CFO', '孟晚舟', '於當', '地', '時間', '12月', '1日', '在', '加拿大', '被捕']
['全球', '最大', '電信', '網路設', '備製', '造商', '之一', '華為']
['12月', '1日', '在', '加拿大', '被捕']


In [16]:
print(nltk_tree)

(ROOT
  (IP
    (NP
      (CP
        (IP
          (NP
            (NP
              (ADJP (NP (NN 全球)) (ADJP (JJ 最大)))
              (NP (NN 電信) (NN 網路設)))
            (NP (NN 備製) (NN 造商) (NN 之一)))
          (VP (VV 華為)))
        (DEC 的))
      (NP (JJ 首席) (NN 財務官))
      (NP (NR CFO) (NR 孟晚舟)))
    (VP
      (DVP (VP (VV 於當)) (DEV 地))
      (VP
        (VV 時間)
        (IP
          (VP
            (NP (NT 12月) (NT 1日))
            (PP (P 在) (NP (NR 加拿大)))
            (VP (VV 被捕))))))))


In [79]:
help(nltk_tree)

Help on Tree in module nltk.tree object:

class Tree(builtins.list)
 |  Tree(node, children=None)
 |  
 |  A Tree represents a hierarchical grouping of leaves and subtrees.
 |  For example, each constituent in a syntax tree is represented by a single Tree.
 |  
 |  A tree's children are encoded as a list of leaves and subtrees,
 |  where a leaf is a basic (non-tree) value; and a subtree is a
 |  nested Tree.
 |  
 |      >>> from nltk.tree import Tree
 |      >>> print(Tree(1, [2, Tree(3, [4]), 5]))
 |      (1 2 (3 4) 5)
 |      >>> vp = Tree('VP', [Tree('V', ['saw']),
 |      ...                  Tree('NP', ['him'])])
 |      >>> s = Tree('S', [Tree('NP', ['I']), vp])
 |      >>> print(s)
 |      (S (NP I) (VP (V saw) (NP him)))
 |      >>> print(s[1])
 |      (VP (V saw) (NP him))
 |      >>> print(s[1,1])
 |      (NP him)
 |      >>> t = Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
 |      >>> s == t
 |      True
 |      >>> t[1][1].set_label('X')
 |      >>> t[1][1].label()


In [17]:
words = list(w for w in nltk_tree.leaves())
position = words.index("孟晚舟")
treeposition = nltk_tree.leaf_treeposition(position)
print(nltk_tree[treeposition[:-1]].leaves())
print(nltk_tree[treeposition[:-1]].label())

['孟晚舟']
NR


In [25]:
for i in range(len(treeposition)):
    print(nltk_tree[treeposition[:i]].leaves())

['全球', '最大', '電信', '網路設', '備製', '造商', '之一', '華為', '的', '首席', '財務官', 'CFO', '孟晚舟', '於當', '地', '時間', '12月', '1日', '在', '加拿大', '被捕']
['全球', '最大', '電信', '網路設', '備製', '造商', '之一', '華為', '的', '首席', '財務官', 'CFO', '孟晚舟', '於當', '地', '時間', '12月', '1日', '在', '加拿大', '被捕']
['全球', '最大', '電信', '網路設', '備製', '造商', '之一', '華為', '的', '首席', '財務官', 'CFO', '孟晚舟']
['CFO', '孟晚舟']
['孟晚舟']
